In [2]:
import sys

import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.aggregators as ag
import entropy.data.cleaners as cl
import entropy.data.make_data as md
import entropy.data.selectors as sl
import entropy.data.validators as vl
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [3]:
df = hd.read_txn_data("777")
hd.inspect(df)

Time for read_txn_data                 : 3.12 seconds
(651,789, 18)


,date,user_id,amount,desc,merchant,tag_group,tag,account_id,account_last_refreshed,account_provider,account_type,debit,female,id,latest_balance,postcode,tag_auto,yob
0,2012-02-01,777,400.00,<mdbremoved> - s/o,NaN,transfers,other_transfers,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688300,364.220001,wa1 4,transfers,1969.0
1,2012-02-01,777,3.03,aviva pa - d/d,aviva,spend,finance,262916,2020-07-21 20:32:00,natwest bank,current,True,0.0,688299,364.220001,wa1 4,health insurance,1969.0


In [321]:
dfa = hd.read_analysis_data("777")
hd.inspect(dfa)

Time for read_analysis_data            : 0.55 seconds
(729, 38)


,user_id,month,date,active_accounts,txns_count,txns_value,txn_count_sa,txn_count_ca,prop_credit,month_income,year_income,has_regular_income,has_month_income,entropy,entropyz,age,female,sa_inflows,has_sa_inflows,has_reg_sa_inflows,has_benefits,has_pension,has_mortgage_pmt,has_rent_pmt,loan_funds,loan_repmt,pdloan_funds,pdloan_repmt,spend_finance,spend_services,spend_other_spend,spend_household,spend_motor,spend_communication,spend_retail,spend_travel,spend_hobbies,month_spend
0,13777,11,2012-11-30,"[112648, 112650, 112652, 112651]",100,317126.781250,14.0,85.0,0.102756,6366.030273,69899.824219,0,1,0.827491,0.081098,43.0,0.0,50076.0,1,0,0,0,3.0,3.0,0,0,0,0,1151.430054,403.660004,1686.0,6983.815918,0.000000,0.0,501.670013,136.899994,25.000000,11073.820312
1,13777,12,2012-12-31,"[112648, 112650, 112652, 112651, 112653]",65,34068.792969,8.0,52.0,0.022894,5283.939941,69899.824219,0,1,0.853535,0.389008,43.0,0.0,3250.0,1,0,0,0,3.0,1.0,0,0,0,0,13.530000,141.889999,1030.0,5677.649902,202.669998,0.0,183.949997,0.000000,94.989998,7344.680176


In [ ]:
df.select_dtypes

In [28]:
d.groupby(["user_id"]).logins.sum().describe()

count     279.000000
mean      127.691756
std       647.330060
min         0.000000
25%         0.000000
50%         4.000000
75%        29.500000
max      7953.000000
Name: logins, dtype: float64